In [30]:
!kaggle datasets download -d khan1803115/eeg-dataset-for-emotion-recognition
!mkdir -p datasets/khan1803115
!unzip eeg-dataset-for-emotion-recognition.zip -d datasets/khan1803115/
!rm eeg-dataset-for-emotion-recognition.zip

Dataset URL: https://www.kaggle.com/datasets/khan1803115/eeg-dataset-for-emotion-recognition
License(s): Apache 2.0
 99% 180M/181M [00:05<00:00, 26.1MB/s]
100% 181M/181M [00:05<00:00, 37.0MB/s]
Archive:  eeg-dataset-for-emotion-recognition.zip
replace datasets/khan1803115/Data/S01G1AllChannels.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [31]:
import os

import numpy as np
import pandas as pd
!pip install mne
import mne

## Data Loading

In [20]:
khan_dataset_dir = "datasets/khan1803115/"
valence = pd.read_csv(khan_dataset_dir+"valence_normalize.csv")
arousal = pd.read_csv(khan_dataset_dir+"arousal_normalize.csv")
data = [pd.read_csv(khan_dataset_dir+"Data/"+file) for file in os.listdir(khan_dataset_dir+"Data/")]

channels = data[0].columns[:-2]
label_columns = data[0].columns[-2:]
fs = 128

In [34]:
eeg_data = np.array([data_[channels][:38000] for data_ in data])
valence_arousal_values = np.array([data_[label_columns].mean() for data_ in data])
eeg_data.shape, valence_arousal_values.shape

((100, 38000, 14), (100, 2))

In [35]:
# Make use of 1250 samples for an epoch with 500 samples overlapping between succesive epochs
epoch_starting_points = np.arange(0,38252,1280-256)[:-4]

eeg_epochs = []

for i in range(len(data)):
    epochs = []
    for epoch_starting_point in epoch_starting_points:
        epochs.append(eeg_data[i,epoch_starting_point:epoch_starting_point+1280,:])

    eeg_epochs.append(epochs)

eeg_epochs = np.array(eeg_epochs)

eeg_epochs.shape

(100, 34, 1280, 14)

In [36]:
epoch_targets = []

for i in range(len(data)):
  targets = [valence_arousal_values[i]]*34
  epoch_targets.append(targets)

epoch_targets = np.array(epoch_targets)

epoch_targets.shape

(100, 34, 2)

In [38]:
eeg_epochs = eeg_epochs.reshape(3400,1280,14)
epoch_targets = epoch_targets.reshape(3400,2)

eeg_epochs.shape, epoch_targets.shape

((3400, 1280, 14), (3400, 2))

## Preprocessing

In [ ]:
lowcut = 1
highcut = 45

filtered_eeg_epochs = np.empty((3400,1280,14))

for e,epoch in enumerate(eeg_epochs):
  filtered = mne.filter.filter_data(epoch.T,fs,lowcut,highcut,verbose=0)
  filtered_eeg_epochs[e,:,:] = filtered.T